# Multi-Lingual BERT - reliability

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

data = pd.read_excel("SQP_database_for_meta_analysis_20240111.xlsx")
data = data.dropna(subset=["reliability(r^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["reliability(r^2)"]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

class QualityDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDataset(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_labels.tolist())
test_dataset = QualityDataset(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_labels.tolist())




Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 0
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=1)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()


# Random Forest - reliability

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
import pandas as pd

data = pd.read_excel("SQP_database_for_meta_analysis_20240111.xlsx")
data = data.dropna(subset=['reliability(r^2)'])
data.fillna(-1, inplace=True)

domain_index = data.columns.get_loc("Domain")
position_index = data.columns.get_loc("Position")

features = data.iloc[:, domain_index:position_index + 1]  # Including 'Position' column
target = data['reliability(r^2)']

features.info()
missing_values = features.isnull().sum().sum()
target_missing = target.isnull().sum()

print(missing_values)




In [ ]:
# features_clean = data_clean.iloc[:, domain_index:position_index + 1]
# target_clean = data_clean['reliability(r^2)']

# # Imputing missing values in features
# imputer = SimpleImputer(strategy='median')
# features_imputed = imputer.fit_transform(features_clean)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

mse


In [ ]:
train_predictions = rf_model.predict(X_train)
train_mse = mean_squared_error(y_train, train_predictions)

train_mse

# Multi-Lingual BERT - validity

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

data = pd.read_excel("SQP_database_for_meta_analysis_20240111.xlsx")
data = data.dropna(subset=["validity(v^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["validity(v^2)"]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

class QualityDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDataset(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_labels.tolist())
test_dataset = QualityDataset(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_labels.tolist())




Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 0
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=1)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()


# Random Forest - validity

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
import pandas as pd

data = pd.read_excel("SQP_database_for_meta_analysis_20240111.xlsx")
data = data.dropna(subset=['validity(v^2)'])
data.fillna(-1, inplace=True)

domain_index = data.columns.get_loc("Domain")
position_index = data.columns.get_loc("Position")

features = data.iloc[:, domain_index:position_index]  # Including 'Position' column
target = data['validity(v^2)']

features.info()
missing_values = features.isnull().sum().sum()
target_missing = target.isnull().sum()

print(missing_values)




In [ ]:
# features_clean = data_clean.iloc[:, domain_index:position_index + 1]
# target_clean = data_clean['reliability(r^2)']

# # Imputing missing values in features
# imputer = SimpleImputer(strategy='median')
# features_imputed = imputer.fit_transform(features_clean)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

mse


In [ ]:
train_predictions = rf_model.predict(X_train)
train_mse = mean_squared_error(y_train, train_predictions)

train_mse

# Multi-Lingual BERT - quality

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

data = pd.read_excel("SQP_database_for_meta_analysis_20240111.xlsx")
data = data.dropna(subset=["quality(q^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

class QualityDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDataset(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_labels.tolist())
test_dataset = QualityDataset(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_labels.tolist())




Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=1)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()


# Random Forest - quality

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
import pandas as pd

data = pd.read_excel("SQP_dummyvars_data.xlsx")
# data = data.dropna(subset=['quality(q^2)'])
data.fillna(-1, inplace=True)

domain_index = data.columns.get_loc("Domain_ori")
position_index = data.columns.get_loc("Position_ori")
last_external_index = data.columns.get_loc("Showcard or other visual aids used_ori")

features = data.iloc[:, domain_index:last_external_index] 
target = data['quality(q^2)']

features.info()
# missing_values = features.isnull().sum().sum()
# target_missing = target.isnull().sum()

# print(missing_values)




In [ ]:
# features_clean = data_clean.iloc[:, domain_index:position_index + 1]
# target_clean = data_clean['reliability(r^2)']

# # Imputing missing values in features
# imputer = SimpleImputer(strategy='median')
# features_imputed = imputer.fit_transform(features_clean)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

mse

In [ ]:
# features_clean = data_clean.iloc[:, domain_index:position_index + 1]
# target_clean = data_clean['reliability(r^2)']

# # Imputing missing values in features
# imputer = SimpleImputer(strategy='median')
# features_imputed = imputer.fit_transform(features_clean)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

mse


In [ ]:
X_test.index[:50]

In [ ]:
X_test.index[23]

In [ ]:
import numpy as np
residuals_train = y_train - rf_model.predict(X_train)
residuals_test = y_test - predictions

std_dev_train = np.std(residuals_train)
std_dev_test = np.std(residuals_test)

threshold_train = 3 * std_dev_train
threshold_test = 3 * std_dev_test

outliers_train = np.where(np.abs(residuals_train) > threshold_train)[0]
print("Training set outliers:", outliers_train)

outliers_test = np.where(np.abs(residuals_test) > threshold_test)[0]
print("Test set outliers:", outliers_test)


In [ ]:
train_indices = X_train.index
test_indices = X_test.index

outlier_indices_train = train_indices[outliers_train]
outlier_indices_test = test_indices[outliers_test]

outlier_details_train = data.loc[outlier_indices_train, :]
outlier_details_test = data.loc[outlier_indices_test, :]

# print("Training set outlier details:")
# print(outlier_details_train)
# print("\nTest set outlier details:")
# print(outlier_details_test)
outlier_details_train.to_csv("outliers/rf_quality_train.csv")
outlier_details_test.to_csv("outliers/rf_quality_test.csv")

In [ ]:
outlier_details_train['Language'].value_counts()

In [ ]:
outlier_details_test['Language'].value_counts()

In [ ]:
print("\nTest set outlier details:")
print(outlier_details_test)

In [ ]:
unique_languages = data['Language'].unique()
print(unique_languages)

In [ ]:
slovene_count = data[data['Language'] == 'Slovene'].shape[0]
print(slovene_count)


In [ ]:
train_predictions = rf_model.predict(X_train)
train_mse = mean_squared_error(y_train, train_predictions)

train_mse

# BERT with additional features

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

data = pd.read_excel("SQP_database_for_meta_analysis_20240111.xlsx")
data = data.dropna(subset=["validity(v^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["validity(v^2)"]


data.fillna(-1, inplace=True)

domain_index = data.columns.get_loc("Domain")
position_index = data.columns.get_loc("Position")

additional_features = data.iloc[:, domain_index:position_index + 1] 

combined_features = pd.concat([features, additional_features], axis=1)


train_combined, test_combined, train_labels, test_labels = train_test_split(combined_features, labels, test_size=0.2, random_state=42)
num_additional_features = position_index - domain_index + 1


train_features = train_combined.iloc[:, :-num_additional_features]
train_additional_features = train_combined.iloc[:, -num_additional_features:]


test_features = test_combined.iloc[:, :-num_additional_features]
test_additional_features = test_combined.iloc[:, -num_additional_features:]







In [ ]:
from transformers import BertModel
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

class QualityDatasetWithFeatures(Dataset):
    def __init__(self, tokenizer, texts, additional_features, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128, return_tensors="pt")
        self.additional_features = additional_features
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['additional_features'] = torch.tensor(self.additional_features.iloc[idx], dtype=torch.float)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, bert_model_name, additional_feature_dim, num_labels):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.additional_feature_dim = additional_feature_dim
        self.num_labels = num_labels
        
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + additional_feature_dim, num_labels)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        additional_features=None):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         pooled_output = outputs.pooler_output
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        combined_features = torch.cat((pooled_output, additional_features), 1)
        logits = self.classifier(combined_features)
        
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDatasetWithFeatures(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_additional_features, train_labels.tolist())
test_dataset = QualityDatasetWithFeatures(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_additional_features, test_labels.tolist())

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

# BERT with additional features - residual learning

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

data = pd.read_excel("SQP_database_for_meta_analysis_20240111.xlsx")
data = data.dropna(subset=["validity(v^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["validity(v^2)"]


data.fillna(-1, inplace=True)

domain_index = data.columns.get_loc("Domain")
position_index = data.columns.get_loc("Position")

additional_features = data.iloc[:, domain_index:position_index + 1] 

combined_features = pd.concat([features, additional_features], axis=1)


train_combined, test_combined, train_labels, test_labels = train_test_split(combined_features, labels, test_size=0.2, random_state=42)
num_additional_features = position_index - domain_index + 1


train_features = train_combined.iloc[:, :-num_additional_features]
train_additional_features = train_combined.iloc[:, -num_additional_features:]


test_features = test_combined.iloc[:, :-num_additional_features]
test_additional_features = test_combined.iloc[:, -num_additional_features:]







In [ ]:
from transformers import BertModel
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

class QualityDatasetWithFeatures(Dataset):
    def __init__(self, tokenizer, texts, additional_features, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128, return_tensors="pt")
        self.additional_features = additional_features
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['additional_features'] = torch.tensor(self.additional_features.iloc[idx], dtype=torch.float)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, bert_model_name, additional_feature_dim, num_labels):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.additional_feature_dim = additional_feature_dim
        self.num_labels = num_labels
        
        self.dropout = nn.Dropout(0.1)
        self.dense = nn.Linear(768 + additional_feature_dim, 768)
        self.classifier = nn.Linear(768, num_labels)
        self.relu = nn.ReLU()

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        additional_features=None):
        
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
#         pooled_output = outputs.pooler_output
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        combined_features = torch.cat((pooled_output, additional_features), 1)
        logits = self.relu(self.dense(combined_features))
        fxplusx = logits+pooled_output
        logits = self.classifier(fxplusx)
        
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDatasetWithFeatures(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_additional_features, train_labels.tolist())
test_dataset = QualityDatasetWithFeatures(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_additional_features, test_labels.tolist())

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=500,
#     lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=500,
#     lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

In [ ]:
train_additional_features.iloc[0:2]

In [ ]:
a+a

# Quality

# BERT with additional features

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

data = pd.read_excel("SQP_database_for_meta_analysis_20240111.xlsx")
data = data.dropna(subset=["quality(q^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]


data.fillna(-1, inplace=True)

domain_index = data.columns.get_loc("Domain")
position_index = data.columns.get_loc("Position")

additional_features = data.iloc[:, domain_index:position_index + 1] 

combined_features = pd.concat([features, additional_features], axis=1)


train_combined, test_combined, train_labels, test_labels = train_test_split(combined_features, labels, test_size=0.2, random_state=42)
num_additional_features = position_index - domain_index + 1


train_features = train_combined.iloc[:, :-num_additional_features]
train_additional_features = train_combined.iloc[:, -num_additional_features:]


test_features = test_combined.iloc[:, :-num_additional_features]
test_additional_features = test_combined.iloc[:, -num_additional_features:]







In [ ]:
from transformers import BertModel
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

class QualityDatasetWithFeatures(Dataset):
    def __init__(self, tokenizer, texts, additional_features, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128, return_tensors="pt")
        self.additional_features = additional_features
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['additional_features'] = torch.tensor(self.additional_features.iloc[idx], dtype=torch.float)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, bert_model_name, additional_feature_dim, num_labels):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.additional_feature_dim = additional_feature_dim
        self.num_labels = num_labels
        
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + additional_feature_dim, num_labels)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        additional_features=None):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         pooled_output = outputs.pooler_output
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        combined_features = torch.cat((pooled_output, additional_features), 1)
        logits = self.classifier(combined_features)
        
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDatasetWithFeatures(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_additional_features, train_labels.tolist())
test_dataset = QualityDatasetWithFeatures(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_additional_features, test_labels.tolist())

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

# BERT with NN and additional dummy features without Position

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

# Load the processed data
data = pd.read_excel('SQP_dummyvars_data.xlsx')

# Define numerical features
numerical_features = [
    'Number of categories', 'Maximum possible value_75', 'Maximum possible value_76',
    'Number of fixed reference points', 'Number of sentences in introduction',
    'Number of words in introduction', 'Number of subordinated clauses in introduction',
    'Number of sentences in the request', 'Number of words in request',
    'Total number of nouns in request for an answer',
    'Total number of abstract nouns in request for an answer',
    'Total number of syllables in request', 'Number of subordinate clauses in request',
    'Number of syllables in answer scale', 'Total number of nouns in answer scale',
    'Total number of abstract nouns in answer scale'
]

# Define categorical features
categorical_feature_start = data.columns.get_loc('Domain_1')
categorical_feature_end = data.columns.get_loc('Visual or oral presentation_1') + 1
categorical_features = data.columns[categorical_feature_start:categorical_feature_end]

# Combine all feature columns
additional_features_names = numerical_features + list(categorical_features)


features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]

additional_features = data[additional_features_names] 

combined_features = pd.concat([features, additional_features], axis=1)


train_combined, test_combined, train_labels, test_labels = train_test_split(combined_features, labels, test_size=0.2, random_state=42)
num_additional_features = len(additional_features_names)


train_features = train_combined.iloc[:, :-num_additional_features]
train_additional_features = train_combined.iloc[:, -num_additional_features:]


test_features = test_combined.iloc[:, :-num_additional_features]
test_additional_features = test_combined.iloc[:, -num_additional_features:]







In [ ]:
test_combined.index

In [ ]:
from transformers import BertModel
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

class QualityDatasetWithFeatures(Dataset):
    def __init__(self, tokenizer, texts, additional_features, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128, return_tensors="pt")
        self.additional_features = additional_features
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['additional_features'] = torch.tensor(self.additional_features.iloc[idx], dtype=torch.float)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, bert_model_name, additional_feature_dim, num_labels):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.additional_feature_dim = additional_feature_dim
        self.num_labels = num_labels
        
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + additional_feature_dim, num_labels)
        
        self.fc0 = nn.Linear(768 + additional_feature_dim, 256)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 1)
        
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.15)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        additional_features=None):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         pooled_output = outputs.pooler_output
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        combined_features = torch.cat((pooled_output, additional_features), 1)
#         logits = self.classifier(combined_features)
        x = self.relu(self.fc0(combined_features))
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        logits = self.fc4(x)
        
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDatasetWithFeatures(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_additional_features, train_labels.tolist())
test_dataset = QualityDatasetWithFeatures(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_additional_features, test_labels.tolist())

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

In [ ]:
import numpy as np
# Generate predictions for training and test sets
train_outputs = trainer.predict(train_dataset).predictions.squeeze()
test_outputs = trainer.predict(test_dataset).predictions.squeeze()

# Calculate residuals
residuals_train = train_labels.to_numpy() - train_outputs
residuals_test = test_labels.to_numpy() - test_outputs

# Calculate standard deviation of residuals
std_dev_train = np.std(residuals_train)
std_dev_test = np.std(residuals_test)

# Define thresholds
threshold_train = 3 * std_dev_train
threshold_test = 3 * std_dev_test

# Identify outlier indices
outliers_train = np.where(np.abs(residuals_train) > threshold_train)[0]
outliers_test = np.where(np.abs(residuals_test) > threshold_test)[0]

# Map indices to the original data indices
train_indices = train_combined.index[outliers_train]
test_indices = test_combined.index[outliers_test]

# Retrieve outlier details
outlier_details_train = data.loc[train_indices, :]
outlier_details_test = data.loc[test_indices, :]

# # Export outlier details to CSV
outlier_details_train.to_csv("outliers/mbert_dummy_quality_train.csv")
outlier_details_test.to_csv("outliers/mbert_dummy_quality_test.csv")


In [ ]:
outlier_details_train['Language'].value_counts()

In [ ]:
outlier_details_test['Language'].value_counts()

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

# BERT compressed output with additional dummy features

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

# Load the processed data
data = pd.read_excel('SQP_dummyvars_data.xlsx')

# Define numerical features
numerical_features = [
    'Number of categories', 'Maximum possible value_75', 'Maximum possible value_76',
    'Number of fixed reference points', 'Number of sentences in introduction',
    'Number of words in introduction', 'Number of subordinated clauses in introduction',
    'Number of sentences in the request', 'Number of words in request',
    'Total number of nouns in request for an answer',
    'Total number of abstract nouns in request for an answer',
    'Total number of syllables in request', 'Number of subordinate clauses in request',
    'Number of syllables in answer scale', 'Total number of nouns in answer scale',
    'Total number of abstract nouns in answer scale', 'Position'
]

# Define categorical features
categorical_feature_start = data.columns.get_loc('Domain_1')
categorical_feature_end = data.columns.get_loc('Visual or oral presentation_1') + 1
categorical_features = data.columns[categorical_feature_start:categorical_feature_end]

# Combine all feature columns
additional_features_names = numerical_features + list(categorical_features)


features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]

additional_features = data[additional_features_names] 

combined_features = pd.concat([features, additional_features], axis=1)


train_combined, test_combined, train_labels, test_labels = train_test_split(combined_features, labels, test_size=0.2, random_state=42)
num_additional_features = len(additional_features_names)


train_features = train_combined.iloc[:, :-num_additional_features]
train_additional_features = train_combined.iloc[:, -num_additional_features:]


test_features = test_combined.iloc[:, :-num_additional_features]
test_additional_features = test_combined.iloc[:, -num_additional_features:]







In [ ]:
from transformers import BertModel
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

class QualityDatasetWithFeatures(Dataset):
    def __init__(self, tokenizer, texts, additional_features, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128, return_tensors="pt")
        self.additional_features = additional_features
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['additional_features'] = torch.tensor(self.additional_features.iloc[idx], dtype=torch.float)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, bert_model_name, additional_feature_dim, num_labels):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.additional_feature_dim = additional_feature_dim
        self.num_labels = num_labels
        
        self.dropout = nn.Dropout(0.1)
        self.dense = nn.Linear(768, additional_feature_dim)
        self.classifier = nn.Linear(2 * additional_feature_dim, num_labels)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        additional_features=None):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         pooled_output = outputs.pooler_output
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        compressed_output = self.dense(pooled_output)
        combined_features = torch.cat((compressed_output, additional_features), 1)
        logits = self.classifier(combined_features)
        
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDatasetWithFeatures(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_additional_features, train_labels.tolist())
test_dataset = QualityDatasetWithFeatures(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_additional_features, test_labels.tolist())

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

# BERT with additional dummy features without Position

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

# Load the processed data
data = pd.read_excel('SQP_dummyvars_data.xlsx')

# Define numerical features
numerical_features = [
    'Number of categories', 'Maximum possible value_75', 'Maximum possible value_76',
    'Number of fixed reference points', 'Number of sentences in introduction',
    'Number of words in introduction', 'Number of subordinated clauses in introduction',
    'Number of sentences in the request', 'Number of words in request',
    'Total number of nouns in request for an answer',
    'Total number of abstract nouns in request for an answer',
    'Total number of syllables in request', 'Number of subordinate clauses in request',
    'Number of syllables in answer scale', 'Total number of nouns in answer scale',
    'Total number of abstract nouns in answer scale'
]

# Define categorical features
categorical_feature_start = data.columns.get_loc('Domain_1')
categorical_feature_end = data.columns.get_loc('Visual or oral presentation_1') + 1
categorical_features = data.columns[categorical_feature_start:categorical_feature_end]

# Combine all feature columns
additional_features_names = numerical_features + list(categorical_features)


features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]

additional_features = data[additional_features_names] 

combined_features = pd.concat([features, additional_features], axis=1)


train_combined, test_combined, train_labels, test_labels = train_test_split(combined_features, labels, test_size=0.2, random_state=42)
num_additional_features = len(additional_features_names)


train_features = train_combined.iloc[:, :-num_additional_features]
train_additional_features = train_combined.iloc[:, -num_additional_features:]


test_features = test_combined.iloc[:, :-num_additional_features]
test_additional_features = test_combined.iloc[:, -num_additional_features:]







In [ ]:
test_combined.index

In [ ]:
from transformers import BertModel
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

class QualityDatasetWithFeatures(Dataset):
    def __init__(self, tokenizer, texts, additional_features, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128, return_tensors="pt")
        self.additional_features = additional_features
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['additional_features'] = torch.tensor(self.additional_features.iloc[idx], dtype=torch.float)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, bert_model_name, additional_feature_dim, num_labels):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.additional_feature_dim = additional_feature_dim
        self.num_labels = num_labels
        
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + additional_feature_dim, num_labels)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        additional_features=None):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         pooled_output = outputs.pooler_output
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        combined_features = torch.cat((pooled_output, additional_features), 1)
        logits = self.classifier(combined_features)
        
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDatasetWithFeatures(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_additional_features, train_labels.tolist())
test_dataset = QualityDatasetWithFeatures(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_additional_features, test_labels.tolist())

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertWithAdditionalFeatures('bert-base-multilingual-cased', additional_feature_dim=num_additional_features, num_labels=1)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

# BERT w/o finetune + random forest

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
import pandas as pd

data = pd.read_excel("SQP_dummyvars_data.xlsx")
data = data.dropna(subset=["quality(q^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

class QualityDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = QualityDataset(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_labels.tolist())
test_dataset = QualityDataset(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_labels.tolist())




data = pd.read_excel("SQP_dummyvars_data.xlsx")
# data = data.dropna(subset=['quality(q^2)'])
data.fillna(-1, inplace=True)

domain_index = data.columns.get_loc("Domain_ori")
position_index = data.columns.get_loc("Position_ori")

features = data.iloc[:, domain_index:position_index] 
target = data['quality(q^2)'] 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# rf_model = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
# rf_model.fit(X_train, y_train)

# predictions = rf_model.predict(X_test)
# mse = mean_squared_error(y_test, predictions)

# mse

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
train_dataset[0]

In [ ]:
from transformers import BertModel
import numpy as np
import torch
from tqdm import tqdm

# Check if a GPU is available and if not, use a CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained BERT model and move it to the device (GPU or CPU)
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased').to(device)

# Function to generate embeddings from BERT with tqdm progress bar
def generate_bert_embeddings(dataset):
    bert_model.eval()
    embeddings = []
    for data in tqdm(dataset, desc="Generating BERT embeddings"):
        inputs = {key: val.to(device) for key, val in data.items() if key != 'labels'}
        # Ensure inputs are properly batched
        inputs = {key: val.unsqueeze(0) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
            cls_embeddings = outputs[0][:, 0, :].cpu().numpy()
            embeddings.append(cls_embeddings[0])
    return np.array(embeddings)

# Generate embeddings for both training and testing datasets
train_embeddings = generate_bert_embeddings(train_dataset)
test_embeddings = generate_bert_embeddings(test_dataset)

# Concatenate BERT embeddings with other features. Ensure X_train and X_test are numpy arrays.
X_train_bert = np.concatenate((X_train.values, train_embeddings), axis=1)
X_test_bert = np.concatenate((X_test.values, test_embeddings), axis=1)


# Train Random Forest model on the combined feature set
rf_model_bert = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
rf_model_bert.fit(X_train_bert, y_train)

# Make predictions and calculate MSE
predictions_bert = rf_model_bert.predict(X_test_bert)
mse_bert = mean_squared_error(y_test, predictions_bert)

print("MSE with BERT embeddings:", mse_bert)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}


model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=1)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assume the BERT model is already fine-tuned and we have train_dataset, test_dataset ready

# Function to append BERT predictions to Random Forest features
def append_bert_predictions_to_features(dataset, model, tokenizer, original_features):
    # Generate predictions
    predictions = trainer.predict(dataset)
    
    # Extract the predicted values
    bert_predictions = predictions.predictions.squeeze()
    
    # Ensure alignment of the dataset length and BERT predictions
    assert len(bert_predictions) == len(original_features)
    
    # Concatenate BERT predictions with the original features
    enhanced_features = np.concatenate([original_features, bert_predictions[:, None]], axis=1)
    
    return enhanced_features

# You need to have your original RF features ready in the same order as the train/test datasets
# Assume X_train and X_test are your original RF features corresponding to train_dataset and test_dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Append BERT predictions to RF features
X_train_enhanced = append_bert_predictions_to_features(train_dataset, model, tokenizer, X_train)
X_test_enhanced = append_bert_predictions_to_features(test_dataset, model, tokenizer, X_test)

# Train the Random Forest model with enhanced dataset
rf_model = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
rf_model.fit(X_train_enhanced, y_train)

# Predict and calculate MSE
predictions_rf = rf_model.predict(X_test_enhanced)
mse_rf = mean_squared_error(y_test, predictions_rf)

print("MSE with BERT-enhanced RF model:", mse_rf)


In [ ]:
# Train Random Forest model on the combined feature set
rf_model_bert = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
rf_model_bert.fit(X_train_bert, y_train)

# Make predictions and calculate MSE
predictions_bert = rf_model_bert.predict(X_test_bert)
mse_bert = mean_squared_error(y_test, predictions_bert)

print("MSE with BERT embeddings:", mse_bert)

In [ ]:
# Train Random Forest model on the combined feature set
rf_model_bert2 = RandomForestRegressor(n_estimators=520, max_depth=50, max_features=16, random_state=42)
rf_model_bert2.fit(X_train_bert, y_train)

# Make predictions and calculate MSE
predictions_bert = rf_model_bert2.predict(X_test_bert)
mse_bert = mean_squared_error(y_test, predictions_bert)

print("MSE with BERT embeddings:", mse_bert)

# xlm roberta base

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

data = pd.read_excel("SQP_dummyvars_data.xlsx")
data = data.dropna(subset=["quality(q^2)"])
features = data[["Request for answer text", "Answer options text"]]
# features = data[["Introduction text","Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define a dataset class
class QualityDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
#         new_txt = [str(a)+" "+b for a,b in zip(texts[0],texts[1])]
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Prepare the datasets
train_dataset = QualityDataset(tokenizer, (
#     train_features["Introduction text"].tolist(),
    train_features["Request for answer text"].tolist(), 
    train_features["Answer options text"].tolist()), train_labels.tolist())
test_dataset = QualityDataset(tokenizer, (
#     test_features["Introduction text"].tolist(),
    test_features["Request for answer text"].tolist(), 
    test_features["Answer options text"].tolist()), test_labels.tolist())




In [ ]:
# Training arguments 2025-05-21
training_args = TrainingArguments(
    output_dir='./results_final',
    num_train_epochs=30,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.005,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type='cosine_with_restarts',
    save_total_limit=30,
    save_steps=300,
)

# Define MSE computation for evaluation
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}

# Load the model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

# Start training
trainer.train()

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results_final',
    num_train_epochs=30,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.005,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type='cosine_with_restarts',
    save_total_limit=30,
    save_steps=300,
)

# Define MSE computation for evaluation
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}

# Load the model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

# Start training
trainer.train()

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

checkpoint = "./results_final/checkpoint-5700"

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

training_args = TrainingArguments(
    output_dir='./results_final', 
    logging_dir='./logs',          
    do_train=False,                
    do_eval=False,                 
)

trainer = Trainer(
    model=model,
    args=training_args
)

print(model.config)


In [ ]:


# Use trainer to predict on the test data CSV
predictions = trainer.predict(test_dataset)

# Extract the predicted labels
predicted_labels = predictions.predictions.squeeze()

# Compare predicted labels with actual labels
mse = mean_squared_error(test_labels, predicted_labels)
print(f"Mean Squared Error on the test set: {mse}")

# # Optionally, save predictions to a CSV file
# output_df = test_features_csv.copy()
# output_df['actual_quality'] = test_labels_csv
# output_df['predicted_quality'] = predicted_labels
# output_df.to_csv('./group_results/predictionsESS1 political_efficacy.csv', index=False)

In [ ]:
results_df = pd.DataFrame({
    'y_test': test_labels.values, 
    'predictions': predicted_labels
})

results_csv_path = 'boxplot2.csv' 
results_df.to_csv(results_csv_path, index=False)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type='cosine',
    save_total_limit=1
)

# Define MSE computation for evaluation
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}

# Load the model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

# Start training
trainer.train()

In [ ]:
good_questions=["""Now I have a couple of statements about keeping up with news of current affairs.
“I’m doing a better job now than I was a year ago at keeping up with news about current affairs.” 
“I’m doing a worse job now than I was a year ago at keeping up with news about current affairs.”
Which of these represents your situation, or are you somewhere in between?
 Would you say that you are doing a lot better or just somewhat better at keeping up with the news about current affairs?""",
               """“In the country as a whole, business conditions are better now than they were a year ago.”
“In the country as a whole, business conditions are worse now than they were a year ago.
Which if these represents your opinion, or are you somwhere in between?
Would you say that business conditions are a lot better, or just somewhat better?

               """]

good_answers=["""1. A lot better
2. Somewhat better
3. A bit better
4. Hasn’t changed
5. A bit worse 
6. Somewhat worse 
7. A lot worse""",
              """1. A lot better
2. Somewhat better
3. A bit better
4. Haven’t changed
5. A bit worse
6. Somewhat worse
7. A lot worse
"""
]

In [ ]:
bad_questions=["""Are you doing better at keeping up with the news?""",
               """“Business conditions are better now than they were a year ago.”
“Business conditions are worse now than they were a year ago.
Would you say that business conditions are a lot better, or just somewhat better?

               """]

bad_answers=["""1. A lot better 
2. A lot worse""",
              """1. A lot better
2. 
3. 
4. 
5. 
6. 
7. A lot worse
"""
]

In [ ]:
input_texts = (good_questions, good_answers)

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

predict_dataset = QualityDataset(tokenizer, input_texts,labels=[0,0])
predictions = trainer.predict(predict_dataset)

print(predictions.predictions)

In [ ]:
input_texts = (bad_questions, bad_answers)

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

predict_dataset = QualityDataset(tokenizer, input_texts,labels=[0,0])
predictions = trainer.predict(predict_dataset)

print(predictions.predictions)

In [ ]:
input_texts = (good_questions, bad_answers)

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

predict_dataset = QualityDataset(tokenizer, input_texts,labels=[0,0])
predictions = trainer.predict(predict_dataset)

print(predictions.predictions)

In [ ]:
input_texts = (bad_questions, good_answers)

tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

predict_dataset = QualityDataset(tokenizer, input_texts,labels=[0,0])
predictions = trainer.predict(predict_dataset)

print(predictions.predictions)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type='cosine',
    save_total_limit=1
)

# Define MSE computation for evaluation
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}

# Load the model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

# Start training
trainer.train()

# xlm roberta large

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

data = pd.read_excel("SQP_dummyvars_data.xlsx")
data = data.dropna(subset=["quality(q^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define a dataset class
class QualityDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

# Prepare the datasets
train_dataset = QualityDataset(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_labels.tolist())
test_dataset = QualityDataset(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_labels.tolist())




In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type='cosine',
    save_total_limit=1
)

# Define MSE computation for evaluation
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}

# Load the model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-large', num_labels=1)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

# Start training
trainer.train()

# Multilingual-E5-large

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification

data = pd.read_excel("SQP_dummyvars_data.xlsx")
data = data.dropna(subset=["quality(q^2)"])
features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define a dataset class
class QualityDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')

# Prepare the datasets
train_dataset = QualityDataset(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_labels.tolist())
test_dataset = QualityDataset(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_labels.tolist())




In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type='cosine',
    save_total_limit=1
)

# Define MSE computation for evaluation
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}

# Load the model
model = AutoModelForSequenceClassification.from_pretrained('intfloat/multilingual-e5-large', num_labels=1)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

# Start training
trainer.train()

# BERT 
## with NN
## with additional dummy features 
## with learnable normalized outputs and features
## without Position

In [ ]:
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import mean_squared_error

# Load the processed data
data = pd.read_excel('SQP_dummyvars_data.xlsx')

# Define numerical features
numerical_features = [
    'Number of categories', 'Maximum possible value_75', 'Maximum possible value_76',
    'Number of fixed reference points', 'Number of sentences in introduction',
    'Number of words in introduction', 'Number of subordinated clauses in introduction',
    'Number of sentences in the request', 'Number of words in request',
    'Total number of nouns in request for an answer',
    'Total number of abstract nouns in request for an answer',
    'Total number of syllables in request', 'Number of subordinate clauses in request',
    'Number of syllables in answer scale', 'Total number of nouns in answer scale',
    'Total number of abstract nouns in answer scale'
]

# Define categorical features
categorical_feature_start = data.columns.get_loc('Domain_1')
categorical_feature_end = data.columns.get_loc('Visual or oral presentation_1') + 1
categorical_features = data.columns[categorical_feature_start:categorical_feature_end]

# Combine all feature columns
additional_features_names = numerical_features + list(categorical_features)


features = data[["Request for answer text", "Answer options text"]]
labels = data["quality(q^2)"]

additional_features = data[additional_features_names] 

combined_features = pd.concat([features, additional_features], axis=1)


train_combined, test_combined, train_labels, test_labels = train_test_split(combined_features, labels, test_size=0.2, random_state=42)
num_additional_features = len(additional_features_names)


train_features = train_combined.iloc[:, :-num_additional_features]
train_additional_features = train_combined.iloc[:, -num_additional_features:]


test_features = test_combined.iloc[:, :-num_additional_features]
test_additional_features = test_combined.iloc[:, -num_additional_features:]







In [ ]:
test_combined.index

In [ ]:
from transformers import BertModel
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

class QualityDatasetWithFeatures(Dataset):
    def __init__(self, tokenizer, texts, additional_features, labels=None):
        self.encodings = tokenizer(texts[0], texts[1], truncation=True, padding=True, max_length=128, return_tensors="pt")
        self.additional_features = additional_features
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['additional_features'] = torch.tensor(self.additional_features.iloc[idx], dtype=torch.float)
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, bert_model_name, additional_feature_dim, num_labels):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.additional_feature_dim = additional_feature_dim
        self.num_labels = num_labels
        
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768 + additional_feature_dim, num_labels)
        
        self.norm_dense_bert = nn.Linear(768, 256)
        self.norm_dense_addfeat = nn.Linear(additional_feature_dim, 256)
        
        self.bn_bert = nn.BatchNorm1d(256)
        self.bn_addfeat = nn.BatchNorm1d(256)
        
        self.fc0 = nn.Linear(512, 256)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 1)
        
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.15)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        additional_features=None):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         pooled_output = outputs.pooler_output
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        normalized_pooled_output = self.relu(self.norm_dense_bert(pooled_output))
        normalized_addfeat = self.relu(self.norm_dense_addfeat(additional_features))
        combined_features = torch.cat((normalized_pooled_output, normalized_addfeat), 1)
#         logits = self.classifier(combined_features)
        x = self.relu(self.fc0(combined_features))
        x = self.dropout2(x)
        x = self.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout2(x)
        logits = self.fc4(x)
        
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

train_dataset = QualityDatasetWithFeatures(tokenizer, (train_features["Request for answer text"].tolist(), train_features["Answer options text"].tolist()), train_additional_features, train_labels.tolist())
test_dataset = QualityDatasetWithFeatures(tokenizer, (test_features["Request for answer text"].tolist(), test_features["Answer options text"].tolist()), test_additional_features, test_labels.tolist())

In [ ]:
model = BertWithAdditionalFeatures('xlm-roberta-base', additional_feature_dim=num_additional_features, num_labels=1)
model

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}





trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}





trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
   # logging_dir='./logs',
   # learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=300,
    lr_scheduler_type ='cosine',
    save_total_limit = 1
)

# Compute RMSE
def compute_mse(p):
    return {"mse": mean_squared_error(p.label_ids, p.predictions.squeeze())}





trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_mse,
)

trainer.train()